In [ ]:

from pyspark.sql import functions as F
from pyspark.sql import types as T

# Chemin S3
steam_path = "s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json"

# Lecture du JSON
df_raw = (
    spark.read
    .option("multiLine", "true")
    .json(steam_path)
)

print("Schema brut :")
df_raw.printSchema()

# On "aplatit" la struct `data` -> colonnes simples
df = df_raw.select("data.*")

print("Nombre de jeux :", df.count())
df.printSchema()

display(df.limit(20))



df = df.withColumn(
    "release_date_parsed",
    F.coalesce(
        F.expr("try_to_date(release_date, 'MMM dd, yyyy')"),
        F.expr("try_to_date(release_date, 'dd MMM, yyyy')"),
        F.expr("try_to_date(release_date, 'yyyy-MM-dd')"),
        F.expr("try_to_date(release_date, 'yyyy/M/d')"),
        F.expr("try_to_date(release_date, 'yyyy/MM/dd')")
    )
)

df = df.withColumn("release_year", F.year("release_date_parsed"))

# 2) Prix & discount
# price / initialprice / discount sont des strings.

# Normalisation du texte "free" -> 0
df = df.withColumn(
    "price_clean",
    F.when(F.lower(F.col("price")).isin("free", "free to play"), "0")
     .otherwise(F.col("price"))
)

df = df.withColumn(
    "initialprice_clean",
    F.when(F.lower(F.col("initialprice")).isin("free", "free to play"), "0")
     .otherwise(F.col("initialprice"))
)

# On enlève les symboles / caractères non numériques, puis on utilise try_to_number
df = df.withColumn(
    "price_num_str",
    F.regexp_replace(F.col("price_clean"), "[^0-9.]", "")
)

df = df.withColumn(
    "initial_price_num_str",
    F.regexp_replace(F.col("initialprice_clean"), "[^0-9.]", "")
)

df = df.withColumn(
    "price_final",
    F.expr("try_to_number(price_num_str, '9999999999.99')")
)

df = df.withColumn(
    "initial_price_final",
    F.expr("try_to_number(initial_price_num_str, '9999999999.99')")
)

# Discount : ex "-90%" → 90
df = df.withColumn(
    "discount_num_str",
    F.regexp_replace(F.col("discount"), "[^0-9.]", "")
)

df = df.withColumn(
    "discount_percent",
    F.expr("try_to_number(discount_num_str, '999.99')")
)

# 3) Reviews : positive / negative
df = df.withColumn("positive_reviews", F.col("positive").cast("long"))
df = df.withColumn("negative_reviews", F.col("negative").cast("long"))

df = df.withColumn(
    "total_reviews",
    F.col("positive_reviews") + F.col("negative_reviews")
)

df = df.withColumn(
    "positive_ratio",
    F.when(F.col("total_reviews") > 0,
           F.col("positive_reviews") / F.col("total_reviews"))
     .otherwise(None)
)


df = df.withColumn(
    "owners_clean",
    F.regexp_replace(
        F.regexp_replace(
            F.regexp_replace(
                F.regexp_replace(F.col("owners"), "\\.\\.", "-"),
                "–|—", "-"
            ),
            ",",
            ""
        ),
        " ",
        ""
    )
)

# Maintenant on devrait avoir des formats comme "10000000-20000000"
df = df.withColumn("owners_low_str", F.split(F.col("owners_clean"), "-").getItem(0))
df = df.withColumn("owners_high_str", F.split(F.col("owners_clean"), "-").getItem(1))

df = df.withColumn("owners_low",
                   F.expr("try_to_number(owners_low_str, '999999999999')"))
df = df.withColumn("owners_high",
                   F.expr("try_to_number(owners_high_str, '999999999999')"))

df = df.withColumn(
    "owners_mid",
    F.when(
        F.col("owners_low").isNotNull() & F.col("owners_high").isNotNull(),
        (F.col("owners_low") + F.col("owners_high")) / 2.0
    )
)

# 5) Age requis
df = df.withColumn(
    "required_age_digits",
    F.regexp_extract(F.coalesce(F.col("required_age"), F.lit("")), "(\\d+)", 1)
)

# On convertit proprement en nombre avec try_to_number (ANSI safe)
df = df.withColumn(
    "required_age_int",
    F.expr("try_to_number(required_age_digits, '99')")
)

# 6) Plateformes
df = df.withColumn("is_windows", F.col("platforms.windows").cast("boolean"))
df = df.withColumn("is_mac", F.col("platforms.mac").cast("boolean"))
df = df.withColumn("is_linux", F.col("platforms.linux").cast("boolean"))

display(
    df.select(
        "appid", "name", "publisher",
        "price", "price_final",
        "initialprice", "initial_price_final",
        "discount", "discount_percent",
        "positive_reviews", "negative_reviews", "total_reviews", "positive_ratio",
        "owners", "owners_clean", "owners_low", "owners_high", "owners_mid",
        "release_date", "release_date_parsed", "release_year"
    ).limit(20)
)

top_publishers = (
    df.filter(F.col("publisher").isNotNull())
      .groupBy("publisher")
      .agg(F.count("*").alias("nb_games"))
      .orderBy(F.desc("nb_games"))
)

display(top_publishers)           # vue globale
display(top_publishers.limit(20)) # top 20 éditeurs

min_reviews = 1000

best_rated_games = (
    df.filter((F.col("total_reviews") >= min_reviews) & F.col("positive_ratio").isNotNull())
      .select("appid", "name", "publisher", "developer",
              "total_reviews", "positive_ratio", "price_final", "owners_mid")
      .orderBy(F.desc("positive_ratio"), F.desc("total_reviews"))
)

display(best_rated_games.limit(50))

releases_by_year = (
    df.filter(F.col("release_year").isNotNull())
      .groupBy("release_year")
      .agg(F.count("*").alias("nb_games"))
      .orderBy("release_year")
)

display(releases_by_year)

# Focus 2018–2022
releases_covid_period = releases_by_year.filter(
    (F.col("release_year") >= 2018) & (F.col("release_year") <= 2022)
)

display(releases_covid_period)

# Histogramme des prix
prices = df.select("price_final").filter(F.col("price_final").isNotNull())
display(prices)

# Jeux en promo vs non promo
discount_stats = (
    df.withColumn("has_discount", F.col("discount_percent") > 0)
      .groupBy("has_discount")
      .agg(F.count("*").alias("nb_games"))
)

display(discount_stats)

# Distribution des pourcentages de réduction (pour les jeux en promo)
discount_distribution = (
    df.filter(F.col("discount_percent") > 0)
      .select("discount_percent")
)

display(discount_distribution)

# `languages` est une string : "English, French, Italian"
df_lang = df.withColumn(
    "lang_array",
    F.split(F.coalesce(F.col("languages"), F.lit("")), ",")
)

languages_exploded = (
    df_lang
    .withColumn("language", F.explode("lang_array"))
    .withColumn("language", F.trim(F.lower("language")))
    .filter(F.col("language") != "")
)

top_languages = (
    languages_exploded
    .groupBy("language")
    .agg(F.countDistinct("appid").alias("nb_games"))
    .orderBy(F.desc("nb_games"))
)

display(top_languages.limit(30))


age_distribution = (
    df.groupBy("required_age_int")
      .agg(F.count("*").alias("nb_games"))
      .orderBy("required_age_int")
)

display(age_distribution)

total_games = df.count()
age_16_plus = df.filter(F.col("required_age_int") >= 16).count()
age_18_plus = df.filter(F.col("required_age_int") >= 18).count()

print("Total jeux :", total_games)
if total_games > 0:
    print("Jeux 16+ :", age_16_plus,
          "(" + str(round(age_16_plus / total_games * 100, 2)) + "%)")
    print("Jeux 18+ :", age_18_plus,
          "(" + str(round(age_18_plus / total_games * 100, 2)) + "%)")


df_genres = df.withColumn(
    "genres_array",
    F.split(F.coalesce(F.col("genre"), F.lit("")), ",")
)

df_genres_exploded = (
    df_genres
    .withColumn("genre_clean", F.explode("genres_array"))
    .withColumn("genre_clean", F.trim(F.col("genre_clean")))
    .filter(F.col("genre_clean") != "")
)

display(df_genres_exploded.select("appid", "name", "genre", "genre_clean").limit(20))


genres_count = (
    df_genres_exploded
    .groupBy("genre_clean")
    .agg(F.countDistinct("appid").alias("nb_games"))
    .orderBy(F.desc("nb_games"))
)

display(genres_count)


genre_reviews = (
    df_genres_exploded
    .filter(F.col("total_reviews") >= 100)  # filtre stabilité
    .groupBy("genre_clean")
    .agg(
        F.countDistinct("appid").alias("nb_games"),
        F.avg("positive_ratio").alias("avg_positive_ratio"),
        F.sum("total_reviews").alias("sum_reviews")
    )
    .filter(F.col("nb_games") >= 20)  # on garde les genres un minimum représentés
    .orderBy(F.desc("avg_positive_ratio"))
)

display(genre_reviews)


publisher_genre = (
    df_genres_exploded
    .filter(F.col("publisher").isNotNull())
    .groupBy("publisher", "genre_clean")
    .agg(F.countDistinct("appid").alias("nb_games"))
)

display(publisher_genre.orderBy(F.desc("nb_games")).limit(50))


df_rev = df_genres_exploded.withColumn(
    "revenue_proxy",
    F.when(
        (F.col("price_final").isNotNull()) & (F.col("owners_mid").isNotNull()),
        F.col("price_final") * F.col("owners_mid")
    ).otherwise(F.lit(0.0))
)

genre_revenue = (
    df_rev
    .groupBy("genre_clean")
    .agg(
        F.sum("revenue_proxy").alias("sum_revenue_proxy"),
        F.countDistinct("appid").alias("nb_games")
    )
    .orderBy(F.desc("sum_revenue_proxy"))
)

display(genre_revenue)


platform_stats = (
    df
    .select(
        F.when(F.col("is_windows") == True, 1).otherwise(0).alias("windows"),
        F.when(F.col("is_mac") == True, 1).otherwise(0).alias("mac"),
        F.when(F.col("is_linux") == True, 1).otherwise(0).alias("linux"),
    )
    .agg(
        F.sum("windows").alias("nb_windows"),
        F.sum("mac").alias("nb_mac"),
        F.sum("linux").alias("nb_linux")
    )
)

display(platform_stats)

# Vue "par jeu" avec explode pour compter proprement
df_plat_exploded = (
    df
    .withColumn("platform", F.explode(F.array(
        F.lit("Windows"), F.lit("Mac"), F.lit("Linux")
    )))
    .where(
        (F.col("platform") == "Windows") & (F.col("is_windows") == True) |
        (F.col("platform") == "Mac") & (F.col("is_mac") == True) |
        (F.col("platform") == "Linux") & (F.col("is_linux") == True)
    )
)

platform_counts = (
    df_plat_exploded
    .groupBy("platform")
    .agg(F.countDistinct("appid").alias("nb_games"))
)

display(platform_counts)

df_genre_plat = (
    df
    .join(df_genres_exploded.select("appid", "genre_clean"), on="appid", how="left")
)

genre_platform = (
    df_genre_plat
    .withColumn("platform", F.explode(F.array(
        F.lit("Windows"), F.lit("Mac"), F.lit("Linux")
    )))
    .where(
        (F.col("platform") == "Windows") & (F.col("is_windows") == True) |
        (F.col("platform") == "Mac") & (F.col("is_mac") == True) |
        (F.col("platform") == "Linux") & (F.col("is_linux") == True)
    )
    .groupBy("genre_clean", "platform")
    .agg(F.countDistinct("appid").alias("nb_games"))
)

display(genre_platform.orderBy(F.desc("nb_games")).limit(100))


